In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

Load all the data

In [2]:
files = ['2019-2020.csv', '2020-2021.csv', '2021-2022.csv', 
              '2022-2023.csv', '2023-2024.csv']

df = pd.DataFrame()

for file in files:
    temp = pd.read_csv(file)
    df = pd.concat([df, temp], axis = 0)

print(df.shape)
print(df.head())

(1900, 106)
  Div        Date   Time        HomeTeam          AwayTeam  FTHG  FTAG FTR  \
0  E0  09/08/2019  20:00       Liverpool           Norwich     4     1   H   
1  E0  10/08/2019  12:30        West Ham          Man City     0     5   A   
2  E0  10/08/2019  15:00     Bournemouth  Sheffield United     1     1   D   
3  E0  10/08/2019  15:00         Burnley       Southampton     3     0   H   
4  E0  10/08/2019  15:00  Crystal Palace           Everton     0     0   D   

   HTHG  HTAG  ... AvgC<2.5  AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  \
0     4     0  ...     3.43 -2.25      1.91      1.99   1.94   1.98     1.99   
1     0     1  ...     2.91  1.75      1.95      1.95   1.96   1.97     2.07   
2     0     0  ...     1.92 -0.50      1.95      1.95   1.98   1.95     2.00   
3     0     0  ...     1.71  0.00      1.87      2.03   1.89   2.03     1.90   
4     0     0  ...     1.71  0.25      1.82      2.08   1.97   1.96     2.03   

   MaxCAHA  AvgCAHH  AvgCAHA  
0     2

Understand the variables

In [4]:
variables = pd.DataFrame(columns=['Variable', 'Number of unique values', 'Values'])

for i, var in enumerate(df.columns):
    variables.loc[i] = [var, df[var].nunique, df[var].unique().tolist()]

variables

,Variable,Number of unique values,Values
0,Div,<bound method IndexOpsMixin.nunique of 0 ...,[E0]
1,Date,<bound method IndexOpsMixin.nunique of 0 ...,"[09/08/2019, 10/08/2019, 11/08/2019, 17/08/201..."
2,Time,<bound method IndexOpsMixin.nunique of 0 ...,"[20:00, 12:30, 15:00, 17:30, 14:00, 16:30, 19:..."
3,HomeTeam,<bound method IndexOpsMixin.nunique of 0 ...,"[Liverpool, West Ham, Bournemouth, Burnley, Cr..."
4,AwayTeam,<bound method IndexOpsMixin.nunique of 0 ...,"[Norwich, Man City, Sheffield United, Southamp..."
...,...,...,...
101,PCAHA,<bound method IndexOpsMixin.nunique of 0 ...,"[1.98, 1.97, 1.95, 2.03, 1.96, 1.88, 1.77, 1.8..."
102,MaxCAHH,<bound method IndexOpsMixin.nunique of 0 ...,"[1.99, 2.07, 2.0, 1.9, 2.03, 2.12, 2.21, 2.17,..."
103,MaxCAHA,<bound method IndexOpsMixin.nunique of 0 ...,"[2.07, 1.98, 1.96, 2.08, 1.91, 1.87, 1.97, 1.9..."
104,AvgCAHH,<bound method IndexOpsMixin.nunique of 0 ...,"[1.9, 1.97, 1.96, 1.86, 2.05, 2.08, 2.06, 2.0,..."


### Data Dictionary

|**variable**   |**class**  |**description**|
|---------------|-----------|---------------|
|**Results Data**|
|Div            |String     |League Division|
|Date           |Datetime   |Match Date (dd/mm/yy)|
|Time           |Time       |Time of match kick off|
|HomeTeam       |String     |Home Team|
|AwayTeam       |String     |Away Team|
|FTHG and HG    |int        |Full Time Home Team Goals|
|FTAG and AG    |int        |Full Time Away Team Goals|
|FTR and Res    |String     |Full Time Result (H=Home Win, D=Draw, A=Away Win)|
|HTHG           |int        |Half Time Home Team Goals|
|HTAG           |int        |Half Time Away Team Goals
|HTR            |String     |Half Time Result (H=Home Win, D=Draw, A=Away Win)|


Attendance = Crowd Attendance
Referee = Match Referee
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
HHW = Home Team Hit Woodwork
AHW = Away Team Hit Woodwork
HC = Home Team Corners
AC = Away Team Corners
HF = Home Team Fouls Committed
AF = Away Team Fouls Committed
HFKC = Home Team Free Kicks Conceded
AFKC = Away Team Free Kicks Conceded
HO = Home Team Offsides
AO = Away Team Offsides
HY = Home Team Yellow Cards
AY = Away Team Yellow Cards
HR = Home Team Red Cards
AR = Away Team Red Cards
HBP = Home Team Bookings Points (10 = yellow, 25 = red)
ABP = Away Team Bookings Points (10 = yellow, 25 = red)

B365H = Bet365 home win odds
B365D = Bet365 draw odds
B365A = Bet365 away win odds
BSH = Blue Square home win odds
BSD = Blue Square draw odds
BSA = Blue Square away win odds
BWH = Bet&Win home win odds
BWD = Bet&Win draw odds
BWA = Bet&Win away win odds
GBH = Gamebookers home win odds
GBD = Gamebookers draw odds
GBA = Gamebookers away win odds
IWH = Interwetten home win odds
IWD = Interwetten draw odds
IWA = Interwetten away win odds
LBH = Ladbrokes home win odds
LBD = Ladbrokes draw odds
LBA = Ladbrokes away win odds
PSH and PH = Pinnacle home win odds
PSD and PD = Pinnacle draw odds
PSA and PA = Pinnacle away win odds
SOH = Sporting Odds home win odds
SOD = Sporting Odds draw odds
SOA = Sporting Odds away win odds
SBH = Sportingbet home win odds
SBD = Sportingbet draw odds
SBA = Sportingbet away win odds
SJH = Stan James home win odds
SJD = Stan James draw odds
SJA = Stan James away win odds
SYH = Stanleybet home win odds
SYD = Stanleybet draw odds
SYA = Stanleybet away win odds
VCH = VC Bet home win odds
VCD = VC Bet draw odds
VCA = VC Bet away win odds
WHH = William Hill home win odds
WHD = William Hill draw odds
WHA = William Hill away win odds

Bb1X2 = Number of BetBrain bookmakers used to calculate match odds averages and maximums
BbMxH = Betbrain maximum home win odds
BbAvH = Betbrain average home win odds
BbMxD = Betbrain maximum draw odds
BbAvD = Betbrain average draw win odds
BbMxA = Betbrain maximum away win odds
BbAvA = Betbrain average away win odds

MaxH = Market maximum home win odds
MaxD = Market maximum draw win odds
MaxA = Market maximum away win odds
AvgH = Market average home win odds
AvgD = Market average draw win odds
AvgA = Market average away win odds

Key to total goals betting odds:

BbOU = Number of BetBrain bookmakers used to calculate over/under 2.5 goals (total goals) averages and maximums
BbMx>2.5 = Betbrain maximum over 2.5 goals
BbAv>2.5 = Betbrain average over 2.5 goals
BbMx<2.5 = Betbrain maximum under 2.5 goals
BbAv<2.5 = Betbrain average under 2.5 goals

GB>2.5 = Gamebookers over 2.5 goals
GB<2.5 = Gamebookers under 2.5 goals
B365>2.5 = Bet365 over 2.5 goals
B365<2.5 = Bet365 under 2.5 goals
P>2.5 = Pinnacle over 2.5 goals
P<2.5 = Pinnacle under 2.5 goals
Max>2.5 = Market maximum over 2.5 goals
Max<2.5 = Market maximum under 2.5 goals
Avg>2.5 = Market average over 2.5 goals
Avg<2.5 = Market average under 2.5 goals

Key to Asian handicap betting odds:

BbAH = Number of BetBrain bookmakers used to Asian handicap averages and maximums
BbAHh = Betbrain size of handicap (home team)
AHh = Market size of handicap (home team) (since 2019/2020)
BbMxAHH = Betbrain maximum Asian handicap home team odds
BbAvAHH = Betbrain average Asian handicap home team odds
BbMxAHA = Betbrain maximum Asian handicap away team odds
BbAvAHA = Betbrain average Asian handicap away team odds

GBAHH = Gamebookers Asian handicap home team odds
GBAHA = Gamebookers Asian handicap away team odds
GBAH = Gamebookers size of handicap (home team)
LBAHH = Ladbrokes Asian handicap home team odds
LBAHA = Ladbrokes Asian handicap away team odds
LBAH = Ladbrokes size of handicap (home team)
B365AHH = Bet365 Asian handicap home team odds
B365AHA = Bet365 Asian handicap away team odds
B365AH = Bet365 size of handicap (home team)
PAHH = Pinnacle Asian handicap home team odds
PAHA = Pinnacle Asian handicap away team odds
MaxAHH = Market maximum Asian handicap home team odds
MaxAHA = Market maximum Asian handicap away team odds	
AvgAHH = Market average Asian handicap home team odds
AvgAHA = Market average Asian handicap away team odds